In [15]:
import math
import pickle
import statistics 
import numpy as np
from random import shuffle
from sklearn.metrics import classification_report
from operator import itemgetter


def pickle_operating(fname, item):
    # save or load the pickle file.
    file_name = '%s.pickle' % fname
    print(file_name)
    if not item:
        with open(file_name, 'rb') as fs:
            item = pickle.load(fs)
            return item
    else:
        with open(file_name, 'wb') as fs:
            pickle.dump(item, fs, protocol=pickle.HIGHEST_PROTOCOL)

In [2]:
#the node of the decision tree
class decision_node:
    def __init__(self, feature=-1, val=None, results=None, rb=None, lb=None):
        self.feature = feature 
        self.value = val
        self.results = results 
        self.leftb = lb 
        self.rightb = rb 

In [3]:
# splitting the data based on the value of the specific column
def sets_split(data, column, value):
    split_function = lambda row:row[column] >= value
    set1, set2 = [[], []], [[], []]
    for i in range(len(data[0])): 
        if split_function(data[0][i]):
            set1[0].append(data[0][i])
            set1[1].append(data[1][i])
        else:
            set2[0].append(data[0][i])
            set2[1].append(data[1][i])
    return [set1, set2]

In [4]:
#calculate the entropy from the data source
def entropy(data, labels):
    results_freq = {}
    ent = 0.0
    for i in range(len(data)):
        if labels[i] in results_freq.keys():
            results_freq[labels[i]] += 1
        else:
            results_freq[labels[i]] = 1
    
    for label, freq in results_freq.items():
        ent -= float(freq)/len(data) * math.log(float(freq)/len(data), 2) 
    return ent


#information gaining from this splitting
def info_gain(set1, set2, data_length, current_score):
    p = float(len(set1)) / data_length
    info = current_score - p*entropy(set1[0], set1[1]) - (1-p)*entropy(set2[0], set2[1])
    return info


#make decision on the label/category of the data
def to_decide(records):
    res = {}
    for record in records:
        if record in res:
            res[record] += 1
        else:
            res[record] = 1
    sorted_labels = sorted(res.iteritems(), key=itemgetter(1), reverse=True)
    return sorted_labels[0][0]

In [13]:
#splitting the data based on all of the columns 
#and find the best splitting
def get_best_split(data, split_decision='median'):
    column_count = len(data[0][0])
    
    current_score = entropy(data[0], data[1])
    data_length = len(data[0])
    best = {'score': 0, 'criteria': None, 'sets': None}
    for col in range(0, column_count):
        column_values = [row[col] for row in data[0]]
        if split_decision == 'median':
            value = np.median(column_values)
        else:
            value = np.mean(column_values)
        set1, set2 = sets_split(data, col, value)
        info = info_gain(set1, set2, data_length, current_score)
        if info > best['score'] and len(set1) > 0 and len(set2) > 0:
            best['score'], best['feature'], best['sets'] = info, (col, value), (set1, set2)
    return best

In [20]:
#grow the decision tree recursively
def grow_tree(data, level, max_level=5, split_method='median'):
    if len(data[0]) == 0: 
        return 
    if level >= max_level:
        return decision_node(results=to_decide(data[1]))
    best = get_best_split(data, split_method)
    if best['score'] > 0:
        right_branch = grow_tree(best['sets'][0], level+1, max_level) 
        left_branch = grow_tree(best['sets'][1], level+1, max_level)
        return decision_node(feature=best['feature'][0], val=best['feature'][1],
                             rb=right_branch, lb=left_branch)
    else:
        return decision_node(results=to_decide(data[1]))

In [7]:
#recursive going down the tree to classify the new data point
def predict(node, row):
    if row[node.feature] < node.value:
        if node.leftb.results != None:
            return node.leftb.results
        else:
            return predict(node.leftb, row)
    else:
        if node.rightb.results != None:
            return node.rightb.results
        else:
            return predict(node.rightb, row)

In [11]:
#build the tree on the train data
def experiment(train_data, depth, split_method):
    x_train, y_train  = [x[0] for x in train_data], [x[1] for x in train_data]
    tree = grow_tree([x_train, y_train], 0, depth, split_method)
    pickle_operating('dt_model', tree)
    print("finished growing the tree!")
    return tree

#predicting the test data based on the tree model
#and evaluated the predictions
def predicting(tree, test_data):
    x_test, y_test  = [x[0] for x in test_data], [x[1] for x in test_data]
    predictions = []
    for row in x_test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    print(classification_report(y_test, predictions))
    return predictions

In [67]:
#experimentation runs for Caltech Data after PCA
dataset = pickle_operating('Caltech_data_3', None)
print(len(dataset['train']), len(dataset['test']))
shuffle(dataset['train'])
shuffle(dataset['test'])

Caltech_data_3.pickle
(320, 324)


In [84]:
tree_model = experiment(dataset['train'], 5, 'mean')

dt_model.pickle
finished growing the tree!


In [85]:
predictions = predicting(tree_model, dataset['test'])

             precision    recall  f1-score   support

          1       0.21      0.31      0.25        36
          2       0.16      0.23      0.19        31
          3       0.71      0.45      0.56        33
          4       0.25      0.05      0.08        21
          5       0.34      0.44      0.38        43
          6       0.28      0.39      0.33        46
          7       0.21      0.16      0.18        32
          8       0.11      0.11      0.11        27
          9       0.27      0.16      0.20        25
         10       0.81      0.43      0.57        30

avg / total       0.34      0.30      0.30       324



In [56]:
#experimentation runs for MNIST Data after PCA
dataset = pickle_operating('MNIST_data_2', None)
print(len(dataset['train']), len(dataset['test']))
#randomized the data to avoid the biases
shuffle(dataset['train'])
shuffle(dataset['test'])

MNIST_data_2.pickle
(60000, 10000)


In [57]:
tree_model = experiment(dataset['train'], 15, 'median')
predictions = predicting(tree_model, dataset['test'])

dt_model.pickle
finished growing the tree!
             precision    recall  f1-score   support

          0       0.82      0.90      0.86       980
          1       0.95      0.95      0.95      1135
          2       0.78      0.78      0.78      1032
          3       0.75      0.74      0.74      1010
          4       0.73      0.68      0.70       982
          5       0.77      0.70      0.73       892
          6       0.91      0.86      0.88       958
          7       0.83      0.74      0.78      1028
          8       0.66      0.75      0.70       974
          9       0.65      0.70      0.67      1009

avg / total       0.79      0.78      0.78     10000

